## Case 1 — The Phantom Shipment

**Story:**  
A shipment log looks fishy—some orders appear to have shipped **before** they were even placed. Find all records where `ShipToDate < OrderDate`.

In [2]:
SELECT
    o.OrderID,
    c.CompanyName       AS CustomerCompanyName,
    o.OrderDate,
    o.ShippedDate       AS ShipToDate
FROM Orders AS o
JOIN Customers AS c
  ON o.CustomerID = c.CustomerID
WHERE o.ShippedDate IS NOT NULL
  AND o.ShippedDate < o.OrderDate
ORDER BY o.ShippedDate, o.OrderDate;


(0 rows affected)

Total execution time: 00:00:00.063

OrderID,CustomerCompanyName,OrderDate,ShipToDate


## Case 2 — The Duplicate Address Conspiracy

**Story:**  
Two “different” customers share the exact same address, city, and country — possible shell accounts. List the suspicious pairs.

In [3]:
SELECT
    a.CustomerID   AS CustomerAId,
    a.CompanyName  AS CustomerA,
    b.CustomerID   AS CustomerBId,
    b.CompanyName  AS CustomerB,
    a.Address,
    a.City,
    a.Country
FROM Customers AS a
JOIN Customers AS b
  ON a.CustomerID < b.CustomerID
 AND a.Address = b.Address
 AND a.City    = b.City
 AND a.Country = b.Country
ORDER BY a.Country, a.City, a.Address;


(0 rows affected)

Total execution time: 00:00:00.060

CustomerAId,CustomerA,CustomerBId,CustomerB,Address,City,Country


## Case 3 — The Bulk Buyer

**Story:**  
One-time huge orders are a red flag. Find orders where the total quantity is unusually large (≥ 200 units).

In [4]:
SELECT
    o.OrderID,
    c.CompanyName,
    o.OrderDate,
    SUM(od.Quantity) AS TotalUnits
FROM Orders AS o
JOIN [Order Details] AS od
  ON od.OrderID = o.OrderID
JOIN Customers AS c
  ON c.CustomerID = o.CustomerID
GROUP BY o.OrderID, c.CompanyName, o.OrderDate
HAVING SUM(od.Quantity) >= 200
ORDER BY TotalUnits DESC;


(21 rows affected)

Total execution time: 00:00:00.071

OrderID,CompanyName,OrderDate,TotalUnits
10895,Ernst Handel,1998-02-18 00:00:00.000,346
11030,Save-a-lot Markets,1998-04-17 00:00:00.000,330
10847,Save-a-lot Markets,1998-01-22 00:00:00.000,288
10515,QUICK-Stop,1997-04-23 00:00:00.000,286
10678,Save-a-lot Markets,1997-09-23 00:00:00.000,280
10612,Save-a-lot Markets,1997-07-28 00:00:00.000,263
10990,Ernst Handel,1998-04-01 00:00:00.000,256
10658,QUICK-Stop,1997-09-05 00:00:00.000,255
10845,QUICK-Stop,1998-01-21 00:00:00.000,245
10324,Save-a-lot Markets,1996-10-08 00:00:00.000,241


## Case 4 — The Price Spike

**Story:**  
Some items were sold far above the catalog price. Compare the average sold `UnitPrice` to the catalog `Products.UnitPrice` and surface spikes ≥ 30 %.

In [5]:
SELECT
    p.ProductID,
    p.ProductName,
    p.UnitPrice       AS CatalogPrice,
    AVG(od.UnitPrice) AS AvgSoldPrice,
    (AVG(od.UnitPrice) - p.UnitPrice) / NULLIF(p.UnitPrice,0.0) AS PercentOverList
FROM Products AS p
JOIN [Order Details] AS od
  ON od.ProductID = p.ProductID
GROUP BY p.ProductID, p.ProductName, p.UnitPrice
HAVING AVG(od.UnitPrice) >= 1.30 * p.UnitPrice
ORDER BY PercentOverList DESC;


(0 rows affected)

Total execution time: 00:00:00.046

ProductID,ProductName,CatalogPrice,AvgSoldPrice,PercentOverList


## Case 5 — The Employee Lag

**Story:**  
Which sales employees have the longest average delay between `OrderDate` and `ShippedDate`? Long lags could signal poor workflow—or hidden trouble.

In [6]:
WITH LagCTE AS (
    SELECT
        e.EmployeeID,
        e.FirstName + ' ' + e.LastName AS EmployeeName,
        DATEDIFF(DAY, o.OrderDate, o.ShippedDate) AS DaysToShip
    FROM Employees AS e
    JOIN Orders AS o
      ON o.EmployeeID = e.EmployeeID
    WHERE o.ShippedDate IS NOT NULL
)
SELECT
    EmployeeID,
    EmployeeName,
    AVG(CAST(DaysToShip AS FLOAT)) AS AvgDaysToShip
FROM LagCTE
GROUP BY EmployeeID, EmployeeName
ORDER BY AvgDaysToShip DESC;


(9 rows affected)

Total execution time: 00:00:00.035

EmployeeID,EmployeeName,AvgDaysToShip
9,Anne Dodsworth,10.857142857142858
6,Michael Suyama,9.092307692307692
4,Margaret Peacock,8.821192052980132
8,Laura Callahan,8.68
3,Janet Leverling,8.4251968503937
7,Robert King,8.376811594202898
2,Andrew Fuller,8.053763440860216
1,Nancy Davolio,7.758333333333334
5,Steven Buchanan,7.023809523809524


## Case 6 — The Category Impostors

**Story:**  
Products that sound like beverages aren’t filed under _Beverages_. Identify the misfiled items.

In [7]:
SELECT
    p.ProductName,
    c.CategoryName
FROM Products AS p
JOIN Categories AS c
  ON p.CategoryID = c.CategoryID
WHERE (
       p.ProductName LIKE '%Tea%'
    OR p.ProductName LIKE '%Coffee%'
    OR p.ProductName LIKE '%Juice%'
    OR p.ProductName LIKE '%Soda%'
    OR p.ProductName LIKE '%Milk%'
)
AND c.CategoryName <> 'Beverages'
ORDER BY p.ProductName;


(1 row affected)

Total execution time: 00:00:00.044

ProductName,CategoryName
Teatime Chocolate Biscuits,Confections


## Case 7 — The Silent Supplier

**Story:**  
Some suppliers never appear in any order line. Identify suppliers whose products have _never_ been sold.

In [8]:
SELECT s.SupplierID, s.CompanyName
FROM Suppliers AS s
WHERE NOT EXISTS (
    SELECT 1
    FROM Products AS p
    JOIN [Order Details] AS od
      ON od.ProductID = p.ProductID
    WHERE p.SupplierID = s.SupplierID
)
ORDER BY s.CompanyName;


(0 rows affected)

Total execution time: 00:00:00.031

SupplierID,CompanyName
